# Projet électif python: Comparaison des statitstiques des joueurs de NBA

Dans cette première partie, dédiée aux comparaisons entre joueurs de la NBA, nous utilisons un set de données au format `.csv` tiré du site internet `https://www.nbastuffer.com/`. Ces données proviennent elles-mêmes du site officiel de la NBA mais ont déjà été assemblée dans un fichier regroupant les statistiques globales de chaque joueurs sur la saison 2022-2023.



---


On commence par importer les librairies utilisés pour l'ensemble du projet:


*   numpy pour la gestion des grands nombres ainsi que certaines fonctions mathématiques.
*   panda pour la récupération des données depuis un fichier `.csv`.
*   plotly pour la représentation graphique des statistiques souhaitées avec un contrôle accru de l'infographie.
*   beautifulsoup4 pour la récupération de certaines données issues de pages html

In [33]:
#%pip install pandas numpy plotly requests

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests

#### Récupération des données

Le code ci-dessous permet de récupérer le fichier `.csv` source contenant les stats de l'ensemble des joueurs de la NBA (ici la saison 2022-23) et récupère les informations relatives à un joueur précis.

In [34]:
chemin_fichier_csv = "NBA Stats 202223 All Stats  NBA Player Props Tool.csv"
# Importer les données depuis le fichier CSV
donnees = pd.read_csv(chemin_fichier_csv)
input_joueur = input("Veuillez saisir le nom du joueur : ")
attributs = ['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'TPG', 'TS%']

def trouver_joueur(input_nom, dataframe):
    # Cette fonction recherche le meilleur résultat correspondant à l'input parmi les noms de joueur disponibles dans le dataframe.
    meilleur_resultat = None
    meilleur_score = 0
    # Convertir l'input en minuscule pour la recherche sans distinguer la casse
    input_nom = input_nom.lower()
    
    for joueur in dataframe['NAME']:
        joueur_lower = joueur.lower()
        score = 0
        for mot in input_nom.split():
            if mot in joueur_lower:
                score += 1
        
        # Mettre à jour le meilleur résultat
        if score > meilleur_score:
            meilleur_score = score
            meilleur_resultat = joueur
    
    return meilleur_resultat

def recup_stats_joueur(nom_joueur, attributs):
    stats = {}
    donnees_joueur = donnees[donnees['NAME'] == nom_joueur]
    if not donnees_joueur.empty:
        for attribut in attributs:
            # Filtrer: on retire les valeurs non valides
            valeurs_valides = donnees_joueur[attribut].dropna()
            if not valeurs_valides.empty:
                # Calculer la moyenne des valeurs valides et stocker dans le dictionnaire
                moyenne_attribut = valeurs_valides.mean()
                stats[attribut] = moyenne_attribut
            else:
                print(f"Aucune valeur valide trouvée pour la stat: {attribut}.")
    else:
        print("Le joueur", nom_joueur, "n'a pas été trouvé dans les données.")
    return stats


joueur_trouve = trouver_joueur(input_joueur, donnees)
stats_joueur = recup_stats_joueur(joueur_trouve, attributs)
print(f"Statistiques de {joueur_trouve}:")
for attribut, stat in stats_joueur.items():
    print(f"{attribut}: {stat}")

Statistiques de Stephen Curry:
PPG: 29.4
RPG: 6.1
APG: 6.3
BPG: 0.4
SPG: 0.9
TPG: 3.2
TS%: 0.656


#### Calculs et normalisation des statistiques

Cette fonction prend en entrée un `DataFrame` contenant les données des joueurs ainsi qu'une liste d'attributs.
Elle récupère les données pour chaque joueur dans le `DataFrame`, calcule la moyenne de chaque attribut et renvoie un dictionnaire contenant les moyennes de chaque attribut.

In [35]:
def calculer_moyenne_attributs_joueurs(dataframe, attributs):
    moyennes = {}
    for attribut in attributs:
        valeurs_valides = dataframe[attribut].dropna()
        if len(valeurs_valides) > 0:
            moyenne_attribut = np.mean(valeurs_valides)
            moyennes[attribut] = moyenne_attribut
        else:
            print(f"Aucune valeur valide trouvée pour la stat: {attribut}.")

    return moyennes

# Calcul de la moyenne sur l'ensemble des joueurs de la liste
moyennes_globales = calculer_moyenne_attributs_joueurs(donnees, attributs)
print("Moyennes globales des stats de l'ensemble des joueurs:")
for attribut, moyenne in moyennes_globales.items():
    print(f"{attribut}: {moyenne}")

Moyennes globales des stats de l'ensemble des joueurs:
PPG: 8.950903119868636
RPG: 3.4885057471264367
APG: 2.0320197044334973
BPG: 0.37471264367816093
SPG: 0.6041050903119868
TPG: 1.0809523809523809
TS%: 0.5628811881188118


Cette fonction reprend le principe de la fonction précédente et calcule les moyennes des statistiques pour les joueurs du même poste que celui du joueur spécifié.

In [36]:
def moyenne_attributs_meme_poste(dataframe, nom_joueur, attributs):
    # On n'inclut que les joueurs du même poste
    poste_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'POS'].iloc[0]
    donnees_meme_poste = dataframe[dataframe['POS'] == poste_joueur]
    moyennes_meme_poste = calculer_moyenne_attributs_joueurs(donnees_meme_poste, attributs)
    return moyennes_meme_poste

# Calcul des moyennes des attributs pour les joueurs du même poste que celui du joueur spécifié
moyennes_meme_poste = moyenne_attributs_meme_poste(donnees, joueur_trouve, attributs)
# Afficher les moyennes calculées
print("Moyennes des stats pour les joueurs du même poste que", joueur_trouve)
for attribut, moyenne in moyennes_meme_poste.items():
    print(f"{attribut}: {moyenne}")

Moyennes des stats pour les joueurs du même poste que Stephen Curry
PPG: 9.152083333333334
RPG: 2.5570833333333334
APG: 2.74375
BPG: 0.22958333333333333
SPG: 0.6845833333333334
TPG: 1.19125
TS%: 0.54318410041841


___
On va ensuite effectuer une pondération par le nombre de minute jouées par match afin de réévaluer ces statitistiques sur l'ensemble de la saison et en déduire une moyenne représentative d'un match.

In [37]:
def get_mpg_joueur(dataframe, nom_joueur):
    mpg_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'MPG'].iloc[0]
    return mpg_joueur
get_mpg_joueur(donnees, joueur_trouve)
def get_mpg_joueur_global(dataframe):
    moyenne_mpg = dataframe['MPG'].mean()
    return moyenne_mpg
get_mpg_joueur_global(donnees)
def get_mpg_joueur_poste(dataframe, nom_joueur):
    poste_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'POS'].iloc[0]
    mpg_poste = dataframe.loc[dataframe['POS'] == poste_joueur, 'MPG'].mean()
    return mpg_poste
get_mpg_joueur_poste(donnees, joueur_trouve)       

def ponderation_par_minute(donnees_normalisees_joueur, mpg_joueur):
    donnees_joueur_pond = {attribut: valeur / mpg_joueur for attribut, valeur in donnees_normalisees_joueur.items()}
    return donnees_joueur_pond

ponderation_par_minute(stats_joueur, get_mpg_joueur(donnees, joueur_trouve))
ponderation_par_minute(moyennes_globales, get_mpg_joueur_global(donnees))
ponderation_par_minute(moyennes_meme_poste, get_mpg_joueur_poste(donnees, joueur_trouve))

{'PPG': 0.463064468524687,
 'RPG': 0.12937976978538604,
 'APG': 0.13882447189779484,
 'BPG': 0.01161614032128853,
 'SPG': 0.03463760172028504,
 'TPG': 0.0602732217396804,
 'TS%': 0.0274832786820463}

La fonction ci-dessous permet de normaliser les statistiques récupérées pour le joueur étudié ainsi que les moyennes calculées. Pour ce faire, on recherche la valeur maximale de chaque stat, tous joueurs confondus, puis on calcul le ratio entre ces deux valeurs pour obtenir un pourcentage.
N.B.: ce traitement permet d'aboutir à des stats en % pour obtenir un diagramme homogène.

In [38]:
def normaliser_donnees(dataframe, joueur, attributs, stats):
    # dictionnaire pour stocker les valeurs max
    max_valeurs = {attribut: dataframe[attribut].max() for attribut in attributs}
    # Normaliser les données brutes en pourcentage par rapport à la valeur max
    donnees_normalisees = {}
    for attribut in attributs:
        valeur_joueur = stats[attribut]
        pourcentage_joueur = np.round((valeur_joueur / max_valeurs[attribut]) * 100, 1)
        donnees_normalisees[attribut] = pourcentage_joueur
    return donnees_normalisees

donnees_normalisees_joueur = normaliser_donnees(donnees, joueur_trouve, attributs, stats_joueur)
donnees_normalisees_globales = normaliser_donnees(donnees, joueur_trouve, attributs, moyennes_globales)
donnees_normalisees_poste = normaliser_donnees(donnees, joueur_trouve, attributs, moyennes_meme_poste)

Normalisation avec pondération

In [39]:
donnees_normalisees_pondere_joueur=normaliser_donnees(donnees, joueur_trouve, attributs, ponderation_par_minute(stats_joueur, get_mpg_joueur(donnees, joueur_trouve)))
donnees_normalisees_pondere_globales=normaliser_donnees(donnees, joueur_trouve, attributs, ponderation_par_minute(moyennes_globales, get_mpg_joueur_global(donnees)))
donnees_normalisees_pondere_poste=normaliser_donnees(donnees, joueur_trouve, attributs, ponderation_par_minute(moyennes_meme_poste, get_mpg_joueur_poste(donnees, joueur_trouve)))


---

#### Visualisation du comparatif sous forme de diagramme de Kiviat

La fonction suivante permet la génération d'un diagramme de Kiviat représentant les statistiques normalisées du joueur séléctionné ainsi que celles des joueurs de toute la ligue, et celles des joueurs du même poste (moyennes). L'objectif est de proposer une comparaison pertinente pour un joueur.

**Remarque :** *Les statistiques affichées sont représentées en % et ne représentent donc pas les valeurs réelles.*

In [40]:
def diagramme_kiviat(data_joueur, data_moyenne_globale, data_moyenne_poste, joueur):
    attributs = list(data_joueur.keys())

    valeurs_joueur = list(data_joueur.values())
    valeurs_moyenne_globale = list(data_moyenne_globale.values())
    valeurs_moyenne_poste = list(data_moyenne_poste.values())

    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=valeurs_joueur,
        theta=attributs,
        fill='toself',
        name=f'{joueur}',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatterpolar(
        r=valeurs_moyenne_globale,
        theta=attributs,
        fill='toself',
        name='Moyenne globale',
        line=dict(color='green')
    ))
    fig.add_trace(go.Scatterpolar(
        r=valeurs_moyenne_poste,
        theta=attributs,
        fill='toself',
        name='Moyenne du poste',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title=f"Comparaison des stats pour {joueur}",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]  # Plage de l'axe radial en pourcentage
            )
        )
    )

    fig.show()


diagramme_kiviat(donnees_normalisees_joueur, donnees_normalisees_globales, donnees_normalisees_poste, joueur_trouve)

#### Diagramme de kiviat pondéré

In [41]:
diagramme_kiviat(donnees_normalisees_pondere_joueur, donnees_normalisees_pondere_globales, donnees_normalisees_pondere_poste, joueur_trouve)